# Run Pipeline

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
from time import time
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'wandb'

In [1]:
# Dummy Dataset
class DummyDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.X = np.random.rand(num_samples, 10).astype(np.float32)
        self.y = np.ones(num_samples, dtype=np.float32)  # All labels are 1

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx])

class DummyModel(nn.Module):
    def __init__(self):
        super(DummyModel, self).__init__()
        self.dummy_weight = nn.Parameter(torch.tensor([[1.0]]))  # Learnable scalar

    def forward(self, x):
        batch_size = x.size(0)
        x = self.dummy_weight.expand(batch_size, 1)
        return x

    
# Linear model that ignores learned parameters and always returns 1
class LinearModel1(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=32):
        super(LinearModel1, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        x = self.net(x)  # Compute, but discard the output
        # only_ones = torch.ones(x.size(0), 1, device=x.device)  # Always return 1
        return x
    
# Linear model that ignores learned parameters and always returns 1
class LinearModel2(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=32):
        super(LinearModel2, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 1000),
            nn.ReLU(),
            nn.Linear(1000, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        x = self.net(x)  # Compute, but discard the output
        # only_ones = torch.ones(x.size(0), 1, device=x.device)  # Always return 1
        return x

NameError: name 'Dataset' is not defined

In [ ]:
class Trainer():
    def __init__(self, model, train_loader, test_loader, criterion, optimizer, device):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.total_train_losses = []
        self.total_test_losses = []

    def run(self, epochs=10):
        print(f"### ----- Starting training for {epochs} epochs... ----- ###")
        start = time()
        for epoch in tqdm(range(epochs)):
            self.train(self.model, self.train_loader, self.criterion, self.optimizer, self.device)
            self.test(self.model, self.test_loader, self.criterion, self.device)
        end = time()
        time_taken = end - start
        print(f"Final Train Loss: {self.total_train_losses[-1]} - Final Test Loss: {self.total_test_losses[-1]}")
        print(f"### ----- Training complete. Took {time_taken// 60:.2f} min, {time_taken % 60:.2f} sec. ----- ###")
            
    # Training function with tqdm
    def train(self, model, dataloader, criterion, optimizer, device):
        model.train()
        total_loss = 0.0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * inputs.size(0)
        self.total_train_losses.append(total_loss / len(dataloader.dataset))
        # print(f"Train Loss: {total_loss / len(dataloader.dataset):.4f}")

    # Testing function with tqdm
    def test(self, model, dataloader, criterion, device):
        model.eval()
        total_loss = 0.0
        correct = 0
        with torch.no_grad():
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item() * inputs.size(0)
                predicted = (outputs > 0.5).float()
                correct += (predicted == targets).sum().item()
        accuracy = correct / len(dataloader.dataset)
        self.total_test_losses.append(total_loss / len(dataloader.dataset))
        # print(f"Test Loss: {total_loss / len(dataloader.dataset):.4f}, Accuracy: {accuracy:.2f}")

In [ ]:
class SweepRunner():
    def __init__(self, model_class, train_loader, test_loader, criterion, optimizer_class, device):
        self.model_class = model_class
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer_class = optimizer_class
        self.device = device

    def run(self, epochs=10):
        model = self.model_class()
        optimizer = self.optimizer_class(model.parameters())
        trainer = Trainer(model, self.train_loader, self.test_loader, self.criterion, optimizer, self.device)
        trainer.run(epochs)

In [25]:
train_dataset = DummyDataset(num_samples=800)
test_dataset = DummyDataset(num_samples=200)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

model = LinearModel2().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Included for structure
trainer = Trainer(model, train_loader, test_loader, criterion, optimizer, device)
trainer.run(epochs=10)

Epoch 1/10


Train Loss: 0.4860


Test Loss: 0.2792, Accuracy: 1.00
Epoch 2/10


Train Loss: 0.1765


Test Loss: 0.1077, Accuracy: 1.00
Epoch 3/10


...

Test Loss: 0.0120, Accuracy: 1.00
Epoch 9/10


Train Loss: 0.0107


Test Loss: 0.0101, Accuracy: 1.00
Epoch 10/10


Train Loss: 0.0091


Test Loss: 0.0087, Accuracy: 1.00


In [ ]:
train_dataset = DummyDataset(num_samples=800)
test_dataset = DummyDataset(num_samples=200)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

models = [DummyModel().to(device), LinearModel1().to(device), LinearModel2().to(device)]
criterion = nn.BCEWithLogitsLoss()

for model in models:
    optimizers = [optim.SGD(model.parameters(), lr=0.01), optim.Adam(model.parameters(), lr=0.001)]
    for optimizer in optimizers:
        trainer = Trainer(model, train_loader, test_loader, criterion, optimizer, device)
        trainer.run(epochs=5)


### ----- Starting training for 5 epochs... ----- ###


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 13.81it/s]


Final Train Loss: 0.24714624226093293 - Final Test Loss: 0.24100276827812195
### ----- Training complete. Took 0.00 min, 0.38 sec. ----- ###
### ----- Starting training for 5 epochs... ----- ###


100%|██████████| 5/5 [00:00<00:00,  8.41it/s]


...

Final Train Loss: 0.012575814835727214 - Final Test Loss: 0.009721899926662446
### ----- Training complete. Took 0.00 min, 0.67 sec. ----- ###
### ----- Starting training for 5 epochs... ----- ###


100%|██████████| 5/5 [00:00<00:00,  7.74it/s]


Final Train Loss: 0.028038910329341887 - Final Test Loss: 0.022864044681191444
### ----- Training complete. Took 0.00 min, 0.65 sec. ----- ###
### ----- Starting training for 5 epochs... ----- ###


100%|██████████| 5/5 [00:01<00:00,  3.84it/s]

Final Train Loss: 1.2267228648852323e-05 - Final Test Loss: 8.859227455104701e-06
### ----- Training complete. Took 0.00 min, 1.31 sec. ----- ###
